In [5]:
import tensorflow as tf
import pandas as pd
import pickle
import numpy as np
import pickle
import numpy as np
from IPython import display
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.image import resize, resize_with_pad
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

In [ ]:
class GeneratorPipeline(tf.keras.utils.Sequence):

    def __init__(self, X1, X2, y, batch_size,shuffle=False):
        self.X1 = X1
        self.X2 = X2
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.prepare_training_data()
        self.make_dataset()

# encode image using efficientnetb0
# load encoded image

    def cnn_instantiate(self):
        
        inputs = Input(shape=(500,400,3))
        CNN_model = EfficientNetB0(
        include_top=False, # Whether to include the fully-connected layer at the top of the network
        weights='imagenet', # pre-trained weights on ImageNet
        input_tensor=None,
        input_shape= (500,400,3), # It should have 3 inputs channels
        pooling=None # Optional pooling mode for feature extraction when include_top is False
        )(inputs)
        
        pooling = GlobalAveragePooling2D()(CNN_model)
        cnn_dense = Dense(256, activation='relu')(pooling)
        model = Model(inputs=inputs, outputs=cnn_dense)
        
        img_list_file = '../raw_data/Flickr8k_text/Flickr_8k.trainImages.txt'
        
        img_folder_path = '../raw_data/Flickr8k_text'
        
    def extract_features_to_file(self, img_list_file, img_folder_path, features_file):
    
        features_dict = {}
        img_list = np.loadtxt(img_list_file, dtype=str)

        for image_name in img_list:
            img_path = img_folder_path+image_name
            img = image.load_img(img_path, target_size=(500,400,3))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            features = model.predict(x)

            image_title = image_name.split('.')[0]
            features_dict[image_title] = features

        file = open(features_file,"wb")
        pickle.dump(features_dict, file) 
        file.close()
    
    def prepare_training_data(self):
    
        features_file = '../output/extracted_features/extract_features_6k.pkl'
        file = open(features_file, 'rb')
        features_dict = pickle.load(file)
        file.close()
        
        captions_file = '../output/extracted_features/cap4'
        file = open(captions_file, 'rb')
        x2_captions = pickle.load(file)
        file.close()
        
        cap_img_list = x2_captions[0] # first column with list of repeated image id's
        
        X1 = []
        
        for cap_img in cap_img_list:
            img_feature_matrix = features_dict[cap_img][0]
            X1.append(img_feature_matrix)
            
        X1 = np.array(X1)[:100]
        
        X2 = np.array(x2_captions[1]).astype(np.uint32)[:100]
        
        y = np.array([el[0] if len(el)>0 else vocab_size+1 for el in x2_captions[2][:100]])
        
        data = pd.Dataframe({'X1': X1, 'X2': X2, 'y': y})
        
        return data
                                                                    
    def __getitem__(self,index):

        self.load_image(X1)
#         "../raw_data/Flickr8k_text/......."
        

#         X1 = self.X1[index * self.batch_size:(index + 1) * self.batch_size]
#         X2 = self.X2[index * self.batch_size:(index + 1) * self.batch_size]
#         y = self.y[index * self.batch_size:(index + 1) * self.batch_size]

        return [X1,X2], y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
        
    
    def __len__(self):
        return len(self.X1)//self.batch_size

In [13]:
def cnn_instantiate():
        
        inputs = Input(shape=(500,400,3))
        CNN_model = EfficientNetB0(
        include_top=False, # Whether to include the fully-connected layer at the top of the network
        weights='imagenet', # pre-trained weights on ImageNet
        input_tensor=None,
        input_shape= (500,400,3), # It should have exactly 3 inputs channels
        pooling=None # Optional pooling mode for feature extraction when include_top is False
        )(inputs)
        
        pooling = GlobalAveragePooling2D()(CNN_model)
        cnn_dense = Dense(256, activation='relu')(pooling)
        model = Model(inputs=inputs, outputs=cnn_dense)
        
        img_list_file = '../raw_data/Flickr8k_text/Flickr_8k.trainImages.txt'
        
        img_folder_path = '../raw_data/Flickr8k_text'
        
        print(model.summary())

In [14]:
encode_images()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 500, 400, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 16, 13, 1280)     4049571   
                                                                 
 global_average_pooling2d_3   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 256)               327936    
                                                                 
Total params: 4,377,507
Trainable params: 4,335,484
Non-trainable params: 42,023
_________________________________________________________________
None


In [38]:
def prepare_training_data(self):
    
        features_file = '../output/extracted_features/extract_features_6k.pkl'
        file = open(features_file, 'rb')
        features_dict = pickle.load(file)
        file.close()
        
        captions_file = '../output/extracted_features/cap4'
        file = open(captions_file, 'rb')
        x2_captions = pickle.load(file)
        file.close()
        
        cap_img_list = x2_captions[0] # first column with list of repeated image id's
        
        X1 = []
        
        for cap_img in cap_img_list:
            img_feature_matrix = features_dict[cap_img][0]
            X1.append(img_feature_matrix)
            
        X1 = np.array(X1)[:100]
        
        X2 = np.array(x2_captions[1]).astype(np.uint32)[:100]
        
        y = np.array([el[0] if len(el)>0 else vocab_size+1 for el in x2_captions[2][:100]])
        
        data = pd.Dataframe({'X1': X1, 'X2': X2, 'y': y})
        
        return data

In [41]:
!pwd

/Users/ChrisKarg/code/CMaxK/robo_romeo/notebooks


In [42]:
features_file = '../raw_data/output/extracted_features/extract_features_6k.pkl'
file = open(features_file, 'rb')
features_dict = pickle.load(file)
file.close()

In [43]:
captions_file = '../raw_data/output/extracted_features/cap4'
file = open(captions_file, 'rb')
x2_captions = pickle.load(file)
file.close()

In [44]:
cap_img_list = x2_captions[0] # first column with list of repeated image id's

In [46]:
X1 = []

for cap_img in cap_img_list:
    img_feature_matrix = features_dict[cap_img][0]
    X1.append(img_feature_matrix)

In [ ]:
X1 = np.array(X1)[:100]

In [ ]:
X2 = np.array(x2_captions[1]).astype(np.uint32)[:100]

In [ ]:
y = np.array([el[0] if len(el)>0 else vocab_size+1 for el in x2_captions[2][:100]])

In [ ]:
data = pd.Dataframe({'X1': X1, 'X2': X2, 'y': y})